# Wrangle Data
(Acquire and prepare)

This notebook contain steps needed to reproduce acquisition and preparation phases of data science pipeline. There is a CSV file needed to 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv('inpatient_rehabilitation_facilities_12_2021/Inpatient_Rehabilitation_Facility-Conditions_Dec2021.csv')

In [11]:
df

,CMS Certification Number (CCN),Facility Name,Address Line 1,Address Line 2,City,State,Zip Code,County Name,Phone Number,CMS Region,Condition,Count,Footnote
0,013025,ENCOMPASS HEALTH LAKESHORE REHABILITATION HOSP...,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,All other conditions,346,NaN
1,013025,ENCOMPASS HEALTH LAKESHORE REHABILITATION HOSP...,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,Brain disease or condition (non-traumatic),150,NaN
2,013025,ENCOMPASS HEALTH LAKESHORE REHABILITATION HOSP...,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,Brain injury (traumatic),38,NaN
3,013025,ENCOMPASS HEALTH LAKESHORE REHABILITATION HOSP...,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,Hip or femur fracture,156,NaN
4,013025,ENCOMPASS HEALTH LAKESHORE REHABILITATION HOSP...,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,"Hip or knee replacement, amputation or other b...",329,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10588,713027,SAGE REHAB OF LAFAYETTE,"204 ENERGY PKWY, SUITE B",NaN,LAFAYETTE,LA,70508,Lafayette,(337) 446-4300,6,"Hip or knee replacement, amputation or other b...",Not Available,2.0
10589,713027,SAGE REHAB OF LAFAYETTE,"204 ENERGY PKWY, SUITE B",NaN,LAFAYETTE,LA,70508,Lafayette,(337) 446-4300,6,Nervous system disorder (excluding stroke),Not Available,2.0
10590,713027,SAGE REHAB OF LAFAYETTE,"204 ENERGY PKWY, SUITE B",NaN,LAFAYETTE,LA,70508,Lafayette,(337) 446-4300,6,Spinal cord disease or condition (non-traumatic),Not Available,2.0
10591,713027,SAGE REHAB OF LAFAYETTE,"204 ENERGY PKWY, SUITE B",NaN,LAFAYETTE,LA,70508,Lafayette,(337) 446-4300,6,Spinal cord injury (traumatic),Not Available,2.0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10593 entries, 0 to 10592
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   CMS Certification Number (CCN)  10593 non-null  object 
 1   Facility Name                   10593 non-null  object 
 2   Address Line 1                  10593 non-null  object 
 3   Address Line 2                  639 non-null    object 
 4   City                            10593 non-null  object 
 5   State                           10593 non-null  object 
 6   Zip Code                        10593 non-null  int64  
 7   County Name                     10593 non-null  object 
 8   Phone Number                    10593 non-null  object 
 9   CMS Region                      10593 non-null  int64  
 10  Condition                       10593 non-null  object 
 11  Count                           10593 non-null  object 
 12  Footnote                        

In [12]:
df.describe()

,Zip Code,CMS Region,Footnote
count,10593.000000,10593.000000,4326.000000
mean,53611.919286,5.373832,1.320157
std,26878.006572,2.169934,0.466591
min,674.000000,1.000000,1.000000
25%,32043.000000,4.000000,1.000000
50%,53405.000000,5.000000,1.000000
75%,76901.000000,6.000000,2.000000
max,99508.000000,10.000000,2.000000
